# Hurricane Ike Maximum Water Levels
Compute the maximum water level during Hurricane Ike on a 18 million element triangular mesh storm surge model.  Plot the results using [HoloViz](https://holoviz.org/) TriMesh rendering with Datashader. 

In [ ]:
import xarray as xr
import numpy as np
import pandas as pd
import fsspec

### Read the data using the cloud-friendly zarr data format

In [ ]:
fs = fsspec.filesystem('s3', anon=True, client_kwargs={'endpoint_url': 'https://renc.osn.xsede.org'})

In [ ]:
ds = xr.open_dataset(fs.get_mapper('/rsignellbucket2/esip/adcirc/ike'), engine='zarr', chunks={'time':30})

In [ ]:
ds

In [ ]:
ds['zeta']

How many GB of sea surface height data do we have?

In [ ]:
ds['zeta'].nbytes/1.e9

## Use a Dask Cluster to speed up calculations

### Use a Dask Distributed LocalCluster

In [ ]:
#from dask.distributed import LocalCluster, Client

In [ ]:
#cluster = LocalCluster()
#client = Client(cluster)
#client

In [ ]:
#client.close()

We want to take the maximum over the time dimension.  Let's use a Dask cluster to distribute the memory and compute load, getting our work done faster!

In [ ]:
%run ./Start_Dask_Cluster_Nebari.ipynb
## If this notebook is not being run on Nebari/ESIP, replace the above 
## path name with a helper appropriate to your compute environment.  Examples:
# %run ../environment_set_up/Start_Dask_Cluster_Denali.ipynb
# %run ../environment_set_up/Start_Dask_Cluster_Tallgrass.ipynb

In [ ]:
client

In [ ]:
#client.close()

####  Find the maximum water level at each grid cell over the entire storm
This is the compute intensive step, reading all the elevation data

In [ ]:
%%time
max_var = ds['zeta'].max(dim='time').load()
max_var

### Visualize data on mesh using HoloViz.org tools

In [ ]:
import numpy as np
import geoviews as gv
import hvplot.xarray
import holoviews.operation.datashader as dshade

In [ ]:
dshade.datashade.precompute = True

In [ ]:
v = np.vstack((ds['x'], ds['y'], max_var)).T
verts = pd.DataFrame(v, columns=['x','y','vmax'])
points = gv.operation.project_points(gv.Points(verts, vdims=['vmax']))
tris = pd.DataFrame(ds['element'].values.astype('int')-1, columns=['v0','v1','v2'])

In [ ]:
tiles = gv.tile_sources.OSM

In [ ]:
trimesh = gv.TriMesh((tris, points), label='Hurricane Ike simulated Maximum Water Level (m)')
mesh = dshade.rasterize(trimesh).opts(cmap='turbo', colorbar=True, width=650, height=500)

In [ ]:
tiles * mesh

### Extract a time series at a specified lon, lat location

Because Xarray does not yet understand that `x` and `y` are coordinate variables on this triangular mesh, we create our own simple function to find the closest point. If we had a lot of these, we could use a more fancy tree algorithm.

In [ ]:
ds['zeta']

In [ ]:
# find the indices of the points in (x,y) closest to the points in (xi,yi)
def nearxy(x,y,xi,yi):
    ind = np.ones(len(xi),dtype=int)
    for i in range(len(xi)):
        dist = np.sqrt((x-xi[i])**2+(y-yi[i])**2)
        ind[i] = dist.argmin()
    return ind

In [ ]:
#just offshore of Galveston
lat = 29.1329856
lon = -95.1535041

In [ ]:
ind = nearxy(ds['x'].values,ds['y'].values,[lon], [lat])

In [ ]:
ds['zeta'][:,ind].hvplot(x='time', grid=True)

Be a good citizen and shutdown your cluster if you are done using it

In [ ]:
cluster.shutdown()